In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

WebDriverException: ignored

In [ ]:
file = open("profit_and_loss_BOB.csv", "w", newline='\n')
urls = ['/1', '/2', '/3']
data = dict()
write = csv.writer(file)
for i in urls:
    r = requests.get('https://www.moneycontrol.com/financials/axisbank/profit-lossVI/AB16#AB16'.format(i))
    soup = BeautifulSoup(r.text)
    a = soup.find_all('tr') 
    for i in a:
        if (i.text):
            tds = i.find_all('td')
            tds = [x.text for x in tds][:-1]
            header = tds[0]
            if header != '\xa0' and '' not in tds:
                print(tds)
                write.writerow(tds)
                if(header in data):
                    data[header] += tds[1:]
                else:
                    data[header] = tds[1:]
for i in data:
    temp = [i] + data[i]
    write.writerow(temp)

    #some snippets taken from https://www.red-gate.com/simple-talk/databases/sql-server/bi-sql-server/historical-stock-prices-volumes-python-csv-file/

In [ ]:
options = webdriver.ChromeOptions()
s = Service('C:\Users\Admin.JSWSL-DOL-L0444\Downloads\Selenium\chromedriver')
driver = webdriver.Chrome(service = s, options = options)
driver.get('https://finance.yahoo.com/quote/AXISBANK.NS/history?period1=1595203200&period2=1658275200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true')

import time
time.sleep(50)
import csv
file = open("price-history_.csv", "w")
write = csv.writer(file)
write.writerow(['Date', 'Open', 'High', 'Low', 'Close', 'Adj. Close', 'Volume'])
z = driver.find_elements(By.CSS_SELECTOR, "tr")
for i in z[-1::-1]:
    f = i.text.split(" ")
    try:
        eval(f[2])
        f = [f[0] + " " + f[1] + " " + f[2]] + f[3:]
    except:
        pass
    print(f)
    print(len(f))
    write.writerow(f)
    print("written")
        
# print("Now exporting")
# with open("yahoo-finance-cached.html", 'w') as f:
#     f.write(driver.page_source)

In [ ]:
DATE = 0
OPEN = 1
HIGH = 2
LOW = 3
CLOSE = 4

In [ ]:
data = pd.read_csv('/content/AXISBANK.NS.csv')
data


In [ ]:
def func(n):
    close = data.copy()
    close['func'.format(n)] = close['Close'].rolling(n).mean()
    close.dropna(inplace = True)
    close.head()
    return close
    b= func(10)

def graph(n):
    b = func(n)
    fig3 = make_subplots(specs=[[{"secondary_y": True}]])
    fig3.add_trace(go.Scatter(x=b['Date'],y=b['Close'],name='Price'),secondary_y=False)
    fig3.add_trace(go.Scatter(x=b['Date'],y=b['func'],name='Moving Average'),secondary_y=False)
    # fig3.add_trace(go.Scatter(x=data['Date'],y=data['Volume'],name='Volume'),secondary_y=False)
    fig3.show()

In [ ]:
rows = []
def frows(): 
    with open('/content/AXISBANK.NS.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            z = [row[0]] + [eval(x) for x in row[1:]]
            rows.append(z)

frows()

In [ ]:
def mc_ginley(rows, N):
    mc_indices = [rows[0][CLOSE]]
    for i in rows[1:]:
        prev = mc_indices[-1]
        mc_index = prev + (i[CLOSE] - prev)/(0.6* N*((i[CLOSE]/prev)**4))
        mc_indices.append(mc_index)
    return mc_indices


def graph_mc_ginley(rows, N):
    my = data.copy()
    my['mc'] = mc_ginley(rows, N)
    fig3 = make_subplots(specs=[[{"secondary_y": True}]])
    fig3.add_trace(go.Scatter(x=my['Date'],y=my['Close'],name='Price'),secondary_y=False)
    fig3.add_trace(go.Scatter(x=my['Date'],y=my['mc'],name='McGinley'),secondary_y=False)
    # fig3.add_trace(go.Bar(x=data['Date'],y=data['Volume'],name='Volume'),secondary_y=False) change karte rehne ek saath bahut error arha hai
    fig3.show()


In [ ]:
for I in np.arange(10, 100, 10):
    print(I)
    graph(I)
    graph_mc_ginley(rows, I/2)


In [ ]:
def EMA(data, n, col):
    c = data.copy()
    c['ewm'] = c[col].ewm(span=n,min_periods=0,adjust=False,ignore_na=False).mean()
    return c

In [1]:
def wwma(values, n):
  
    return values.ewm(alpha=1/n, adjust=False).mean()

In [ ]:
def ATR(df, n=14):
    data = df.copy()
    high = data['High']
    low = data['Low']
    close = data['Close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    df['atr'] = atr
    return df

In [ ]:
def PGO(n):
    c = func(n)
    c = ATR(c, n)
    c = EMA(c, n, 'atr')
    c.dropna(inplace=True)
    c['PGO'] = (c['Close'] - c['func'])/c['ewm']
    return c

In [ ]:
c = PGO(5)
fig3 = make_subplots(rows = 2, cols = 1)
fig3.add_trace(go.Scatter(x=c['Date'],y=c['Close'],name='Price'), row = 1, col = 1)
fig3.add_trace(go.Scatter(x=c['Date'],y=c['PGO'],name='PG0'),row = 2, col = 1)
fig3.update_layout(height=900, width=1000, title_text="Price & PGO")

In [ ]:
BANKS_NIFTY_200 = {
    "AXISBANK" : "https://www.moneycontrol.com/financials/axisbank/profit-lossVI/AB16/{}#AB16",
    "HDFCBANK" : "https://www.moneycontrol.com/financials/hdfcbank/profit-lossVI/HDF01/{}#HDF01",
    "ICICIBANK" : "https://www.moneycontrol.com/financials/icicibank/profit-lossVI/ICI02/{}#ICI02",
    "INDUSINDBK" : "https://www.moneycontrol.com/financials/indusindbank/profit-lossVI/IIB/{}#IIB",
    "SBIN" : "https://www.moneycontrol.com/financials/statebankofindia/profit-lossVI/SBI/{}#SBI",
    "YESBANK" : "https://www.moneycontrol.com/financials/yesbank/profit-lossVI/YB/{}#YB"
}



In [ ]:
def scrape(url):
    urls = ['/1', '/2', '/3']
    data = dict()
    r = requests.get(url)
    soup_ = BeautifulSoup(r.text)
    z = soup_.find_all("span", class_="span_price_wrap")
#     print(z)
    try:
        data["price"] = eval(z[0].text)
    except:
        data["price"] = "NOT"
    for i in urls:
        r = requests.get(url.format(i))
        soup = BeautifulSoup(r.text)
        z = soup.find_all('tr') 
        for i in z:
            if (i.text):
                tds = i.find_all('td')
                tds = [x.text for x in tds][:-1]
                for l in range(len(tds)):
                    try:
                        
                        tds[l] = eval(tds[l].replace(",", ""))
                    except:
                        pass
                header = tds[0]
                if "Profit & Loss" in header:
                    if(header in data):
                            data[header] += tds[1:]
                    else:
                        data[header] = tds[1:]
                if header in ["Net Profit / Loss for The Year", "Basic EPS (Rs.)", "Diluted EPS (Rs.)"]:
                    if header != '\xa0' and '' not in tds:
        #                 print(tds)
        #                 write.writerow(tds)
                        if(header in data):
                            data[header] += tds[1:]
                        else:
                            data[header] = tds[1:]
    return data
all_banks = dict()
for i in BANKS_NIFTY_200:
    all_banks[i] =  scrape(BANKS_NIFTY_200[i])

In [ ]:
for i in all_banks:
    print(i)
    for j in all_banks[i]:
        print(j, all_banks[i][j])
    print()

In [ ]:
PE_ratio = {i : all_banks[i]['price']/all_banks[i]['Basic EPS (Rs.)'][0] for i in all_banks} 
EARNINGS_GROWTH = {i : (all_banks[i]['Basic EPS (Rs.)'][0] - all_banks[i]['Basic EPS (Rs.)'][1])/(all_banks[i]['Basic EPS (Rs.)'][1]) for i in all_banks}
EARNINGS_GROWTH = {i: EARNINGS_GROWTH[i]*100 for i in EARNINGS_GROWTH}
EPG = {i: PE_ratio[i]/EARNINGS_GROWTH[i] for i in PE_ratio}
PEG = {}
for i in EPG:
    if EPG[i] >= 0:
        PEG[i] = EPG[i] 

        
PE_R = {}
for i in PE_ratio:
    if PE_ratio[i] >= 0: 
        PE_R[i] = PE_ratio[i]


plt.plot([i for i in PEG], [PEG[i] for i in PEG])
plt.xticks(rotation=45, ha='right')
plt.title("PEG Ratio for Banking Scrips")
plt.show()

plt.plot([i for i in PE_R], [PE_R[i] for i in PE_R])
plt.xticks(rotation = 45, ha = 'right')
plt.title("P/E Ratio for Banking Scrips")
plt.show()
x_ = [i for i in PE_R]
# x_ = [i for i in PEG]

fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(x=x_,y=[PEG[i] for i in PEG],name='PEG'), secondary_y=False)
fig3.add_trace(go.Scatter(x=x_,y=[PE_R[i] for i in PE_R],name='P/E'),secondary_y=True)

fig3.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
x_ = all_banks["AXISBANK"]["Profit & Loss account of Axis Bank (in Rs. Cr.)"][-1::-1]
for i in all_banks:
    fig3.add_trace(go.Scatter(x=x_,y=all_banks[i]["Basic EPS (Rs.)"],name=i),secondary_y=False)

fig3.show()